In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.svm import SVC
from xgboost import XGBRegressor
import lightgbm as lgb
import IPython.core.display as display
import IPython.display
import re

In [132]:
teams = pd.read_csv('WTeams.csv')
seasons = pd.read_csv('WSeasons.csv')
ncaa_tourney_seeds = pd.read_csv('WNCAATourneySeeds.csv')
regular_season_results = pd.read_csv('WRegularSeasonCompactResults.csv')
ncaa_tourney_results = pd.read_csv('WNCAATourneyCompactResults.csv')

In [133]:
ncaa_tourney_seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [134]:
regular_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


In [135]:
regular_season_results = regular_season_results.drop('NumOT', axis=1)
# add score difference
regular_season_results['ScoreGap'] = regular_season_results['WScore'] - regular_season_results['LScore']

In [136]:
regular_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,ScoreGap
0,1998,18,3104,91,3202,41,H,50
1,1998,18,3163,87,3221,76,H,11
2,1998,18,3222,66,3261,59,H,7
3,1998,18,3307,69,3365,62,H,7
4,1998,18,3349,115,3411,35,H,80


In [137]:
# compute number of wins at each season
num_wins = regular_season_results.groupby(['Season','WTeamID']).count()
num_wins = num_wins.reset_index()[['Season','WTeamID','DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

In [138]:
# compute number of loses at each season
num_loses = regular_season_results.groupby(['Season','LTeamID']).count()
num_loses = num_loses.reset_index()[['Season','LTeamID','DayNum']].rename(columns={'DayNum':'NumLoses','LTeamID':'TeamID'})

In [139]:
# compute mean of win gap at each season
gap_wins = regular_season_results.groupby(['Season','WTeamID'])['ScoreGap'].mean().reset_index()
gap_wins = gap_wins.rename(columns={'ScoreGap':'GapWins','WTeamID':'TeamID'})

In [140]:
# compute mean of lose gap at each season
gap_loses = regular_season_results.groupby(['Season','LTeamID'])['ScoreGap'].mean().reset_index()
gap_loses = gap_loses.rename(columns={'ScoreGap':'GapLoses','LTeamID':'TeamID'})

In [141]:
df_features_season_w = regular_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = regular_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

In [142]:
df_features_season = pd.concat([df_features_season_w, df_features_season_l]).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

In [143]:
df_features_season.head()

,Season,TeamID
0,1998,3102
1,1998,3103
2,1998,3104
3,1998,3106
4,1998,3108


In [144]:
df_features_season = df_features_season.merge(num_wins, on=['Season','TeamID'], how='left')
df_features_season = df_features_season.merge(num_loses, on=['Season','TeamID'], how='left')
df_features_season = df_features_season.merge(gap_wins, on=['Season','TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loses, on=['Season','TeamID'], how='left')

In [145]:
df_features_season = df_features_season.fillna(0)
df_features_season.head()

,Season,TeamID,NumWins,NumLoses,GapWins,GapLoses
0,1998,3102,4.0,20.0,15.500000,27.850000
1,1998,3103,11.0,18.0,15.000000,18.611111
2,1998,3104,21.0,9.0,21.619048,5.666667
3,1998,3106,6.0,15.0,8.333333,14.466667
4,1998,3108,12.0,11.0,17.750000,16.636364


In [146]:
df_features_season['WinRatio'] = df_features_season['NumWins']/(df_features_season['NumWins']+df_features_season['NumLoses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLoses'] * df_features_season['GapLoses'])
    / (df_features_season['NumWins'] + df_features_season['NumLoses'])
)
df_features_season.head()

,Season,TeamID,NumWins,NumLoses,GapWins,GapLoses,WinRatio,GapAvg
0,1998,3102,4.0,20.0,15.500000,27.850000,0.166667,-20.625000
1,1998,3103,11.0,18.0,15.000000,18.611111,0.379310,-5.862069
2,1998,3104,21.0,9.0,21.619048,5.666667,0.700000,13.433333
3,1998,3106,6.0,15.0,8.333333,14.466667,0.285714,-7.952381
4,1998,3108,12.0,11.0,17.750000,16.636364,0.521739,1.304348


In [147]:
df_features_season.drop(['NumWins','NumLoses','GapWins','GapLoses'], axis=1, inplace=True)

In [148]:
# tourney results
ncaa_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [149]:
ncaa_tourney_results = ncaa_tourney_results.drop(['NumOT','WLoc'], axis=1)

In [150]:
# replacing DayNum by the corresponding round
def get_round(day):
    round_dic = {137: 0, 138: 0, 139: 1, 140: 1, 141: 2, 144: 3, 145: 3, 146: 4, 147: 4, 148: 4, 151:5, 153: 5, 155: 6}
    try:
        return round_dic[day]
    except:
        print(f'Unknow day : {day}')
    return 0

In [151]:
ncaa_tourney_results['Round'] = ncaa_tourney_results['DayNum'].apply(get_round)

In [152]:
ncaa_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round
0,1998,137,3104,94,3422,46,0
1,1998,137,3112,75,3365,63,0
2,1998,137,3163,93,3193,52,0
3,1998,137,3198,59,3266,45,0
4,1998,137,3203,74,3208,72,0


In [153]:
# Train data
df = ncaa_tourney_results.copy()
# 1.seeds
# SeedW: winnning seed, SeedL: loses seed
df = pd.merge(
    df,
    ncaa_tourney_seeds,
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID'],
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedW'})

df = pd.merge(
    df,
    ncaa_tourney_seeds,
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID'],
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedL'})

In [154]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round,SeedW,SeedL
0,1998,137,3104,94,3422,46,0,X02,X15
1,1998,139,3104,75,3417,74,1,X02,X07
2,1998,137,3112,75,3365,63,0,W03,W14
3,1998,139,3112,94,3438,77,1,W03,W06
4,1998,137,3163,93,3193,52,0,W02,W15


In [155]:
def treat_seed(seed):
    return int(seed[1:])

In [156]:
df['SeedW'] = df['SeedW'].apply(treat_seed)
df['SeedL'] = df['SeedL'].apply(treat_seed)
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round,SeedW,SeedL
0,1998,137,3104,94,3422,46,0,2,15
1,1998,139,3104,75,3417,74,1,2,7
2,1998,137,3112,75,3365,63,0,3,14
3,1998,139,3112,94,3438,77,1,3,6
4,1998,137,3163,93,3193,52,0,2,15


In [157]:
# 2. season stats
df = pd.merge(df,
              df_features_season,
              how='left',
              left_on=['Season', 'WTeamID'],
              right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'GapWins': 'GapWinsW',
    'GapLosses': 'GapLossesW',
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW'
}).drop(columns='TeamID', axis=1)

df = pd.merge(df,
              df_features_season,
              how='left',
              left_on=['Season', 'LTeamID'],
              right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'GapWins': 'GapWinsL',
    'GapLosses': 'GapLossesL',
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
}).drop(columns='TeamID', axis=1)


In [158]:
df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL
0,1998,137,3104,94,3422,46,0,2,15,0.700000,13.433333,0.703704,6.111111
1,1998,139,3104,75,3417,74,1,2,7,0.700000,13.433333,0.703704,7.259259
2,1998,137,3112,75,3365,63,0,3,14,0.777778,12.407407,0.785714,17.821429
3,1998,139,3112,94,3438,77,1,3,6,0.777778,12.407407,0.666667,4.148148
4,1998,137,3163,93,3193,52,0,2,15,0.937500,24.875000,0.714286,5.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,2019,140,3390,72,3140,63,1,2,7,0.875000,14.000000,0.806452,6.451613
1382,2019,145,3390,55,3283,46,3,2,11,0.875000,14.000000,0.718750,10.187500
1383,2019,138,3393,70,3200,49,0,3,14,0.750000,10.312500,0.757576,6.606061
1384,2019,138,3417,89,3397,77,0,6,11,0.625000,5.156250,0.612903,6.096774


In [159]:
# duplicate data
def add_loses_mathes(win_df):
    win_rename = {
        "WTeamID": "TeamIdA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIdB",
        "LScore": "ScoreB",
        "SeedW": "SeedA", 
        "SeedL": "SeedB",
        'WinRatioW' : 'WinRatioA',
        'WinRatioL' : 'WinRatioB',
        'GapAvgW' : 'GapAvgA',
        'GapAvgL' : 'GapAvgB'
     } 
    
    lose_rename = {
        "WTeamID": "TeamIdB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIdA",
        "LScore": "ScoreA",
        "SeedW": "SeedB", 
        "SeedL": "SeedA",
        'GapAvgW' : 'GapAvgB',
        'GapAvgL' : 'GapAvgA',
        'WinRatioW' : 'WinRatioB',
        'WinRatioL' : 'WinRatioA',
     }
    win_df = win_df.copy()
    lose_df = win_df.copy()
    
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    return pd.concat([win_df, lose_df], 0, sort=False)

df = add_loses_mathes(df)

In [160]:
# 3.Differences
df['SeedDiff'] = df['SeedA'] - df['SeedB']
df['WinRatioDiff'] = df['WinRatioA'] - df['WinRatioB']
df['GapAvgDiff'] = df['GapAvgA'] - df['GapAvgB']

In [161]:
df.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,Round,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,SeedDiff,WinRatioDiff,GapAvgDiff
0,1998,137,3104,94,3422,46,0,2,15,0.700000,13.433333,0.703704,6.111111,-13,-0.003704,7.322222
1,1998,139,3104,75,3417,74,1,2,7,0.700000,13.433333,0.703704,7.259259,-5,-0.003704,6.174074
2,1998,137,3112,75,3365,63,0,3,14,0.777778,12.407407,0.785714,17.821429,-11,-0.007937,-5.414021
3,1998,139,3112,94,3438,77,1,3,6,0.777778,12.407407,0.666667,4.148148,-3,0.111111,8.259259
4,1998,137,3163,93,3193,52,0,2,15,0.937500,24.875000,0.714286,5.071429,-13,0.223214,19.803571


In [162]:
df_test = pd.read_csv('WSampleSubmissionStage1.csv')
df_test.head()

,ID,Pred
0,2015_3106_3107,0.5
1,2015_3106_3110,0.5
2,2015_3106_3113,0.5
3,2015_3106_3114,0.5
4,2015_3106_3116,0.5


In [163]:
df_test['Season'] = df_test['ID'].apply(lambda x : x.split('_')[0])
df_test['TeamIdA'] = df_test['ID'].apply(lambda x : x.split('_')[1])
df_test['TeamIdB'] = df_test['ID'].apply(lambda x: x.split('_')[2])
df_test.dtypes

ID          object
Pred       float64
Season      object
TeamIdA     object
TeamIdB     object
dtype: object

In [164]:
ncaa_tourney_seeds['Season'] = ncaa_tourney_seeds['Season'].astype('str')
ncaa_tourney_seeds['TeamID'] = ncaa_tourney_seeds['TeamID'].astype('str')
ncaa_tourney_seeds.dtypes

Season    object
Seed      object
TeamID    object
dtype: object

In [165]:
# test data 
# 1.seeds
df_test = pd.merge(
    df_test,
    ncaa_tourney_seeds,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedA'})

df_test = pd.merge(
    df_test, 
    ncaa_tourney_seeds, 
    how='left', 
    left_on=['Season', 'TeamIdB'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedB'})

In [166]:
df_test['SeedA'] = df_test['SeedA'].apply(treat_seed)
df_test['SeedB'] = df_test['SeedB'].apply(treat_seed)

In [167]:
df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB,SeedA,SeedB
0,2015_3106_3107,0.5,2015,3106,3107,15,13
1,2015_3106_3110,0.5,2015,3106,3110,15,14
2,2015_3106_3113,0.5,2015,3106,3113,15,3
3,2015_3106_3114,0.5,2015,3106,3114,15,11
4,2015_3106_3116,0.5,2015,3106,3116,15,10


In [168]:
# 2.season stats
df_features_season['Season'] = df_features_season['Season'].astype('str')
df_features_season['TeamID'] = df_features_season['TeamID'].astype('str')

df_test = pd.merge(df_test,
              df_features_season,
              how='left',
              left_on=['Season', 'TeamIdA'],
              right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'GapWins': 'GapWinsA',
    'GapLosses': 'GapLossesA',
    'WinRatio': 'WinRatioA',
    'GapAvg': 'GapAvgA'
}).drop(columns='TeamID', axis=1)

df_test = pd.merge(df_test,
              df_features_season,
              how='left',
              left_on=['Season', 'TeamIdB'],
              right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'GapWins': 'GapWinsB',
    'GapLosses': 'GapLossesB',
    'WinRatio': 'WinRatioB',
    'GapAvg': 'GapAvgB',
}).drop(columns='TeamID', axis=1)


In [169]:
df_test['SeedDiff'] = df_test['SeedA'] - df_test['SeedB']
df_test['WinRatioDiff'] = df_test['WinRatioA'] - df_test['WinRatioB']
df_test['GapAvgDiff'] = df_test['GapAvgA'] - df_test['GapAvgB']

In [170]:
df.dtypes

Season            int64
DayNum            int64
TeamIdA           int64
ScoreA            int64
TeamIdB           int64
ScoreB            int64
Round             int64
SeedA             int64
SeedB             int64
WinRatioA       float64
GapAvgA         float64
WinRatioB       float64
GapAvgB         float64
SeedDiff          int64
WinRatioDiff    float64
GapAvgDiff      float64
dtype: object

In [171]:
df_test.dtypes

ID               object
Pred            float64
Season           object
TeamIdA          object
TeamIdB          object
SeedA             int64
SeedB             int64
WinRatioA       float64
GapAvgA         float64
WinRatioB       float64
GapAvgB         float64
SeedDiff          int64
WinRatioDiff    float64
GapAvgDiff      float64
dtype: object

In [172]:
df_test[['Season','TeamIdA','TeamIdB']] = df_test[['Season','TeamIdA','TeamIdB']].astype('int')

In [173]:
df_test.dtypes

ID               object
Pred            float64
Season            int64
TeamIdA           int64
TeamIdB           int64
SeedA             int64
SeedB             int64
WinRatioA       float64
GapAvgA         float64
WinRatioB       float64
GapAvgB         float64
SeedDiff          int64
WinRatioDiff    float64
GapAvgDiff      float64
dtype: object

In [174]:
# target
df['ScoreDiff'] = df['ScoreA'] - df['ScoreB']
df['WinA'] = (df['ScoreDiff'] > 0).astype('int')
df['WinA']

0       1
1       1
2       1
3       1
4       1
       ..
1381    0
1382    0
1383    0
1384    0
1385    0
Name: WinA, Length: 2772, dtype: int64

In [175]:
# modeling
features = [
    'SeedA',
    'SeedB',
    'WinRatioA',
    'GapAvgA',
    'WinRatioB',
    'GapAvgB',
    'SeedDiff',
    'WinRatioDiff',
    'GapAvgDiff'
]

In [176]:
def min_max_scaler(features, df_train, df_test):
    min_train = df_train[features].min()
    max_train = df_train[features].max()
    min_test = df_test[features].min()
    max_test = df_test[features].max()
    
    df_train[features] = (df_train[features] - min_train) / (max_train - min_train)
    
    if df_test is not None:
        df_test[features] = (df_test[features] - min_test) / (max_test - min_test)
        
    return df_train, df_test 

In [177]:
df_train, df_test = min_max_scaler(features,df, df_test)

In [178]:
df_train.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,Round,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,SeedDiff,WinRatioDiff,GapAvgDiff,ScoreDiff,WinA
0,1998,137,3104,94,3422,46,0,0.066667,0.933333,0.470588,0.399273,0.477124,0.245620,0.066667,0.496623,0.587422,48,1
1,1998,139,3104,75,3417,74,1,0.066667,0.400000,0.470588,0.399273,0.477124,0.269713,0.333333,0.496623,0.573714,1,1
2,1998,137,3112,75,3365,63,0,0.133333,0.866667,0.607843,0.377744,0.621849,0.491354,0.133333,0.492764,0.435361,12,1
3,1998,139,3112,94,3438,77,1,0.133333,0.333333,0.607843,0.377744,0.411765,0.204428,0.400000,0.601307,0.598609,17,1
4,1998,137,3163,93,3193,52,0,0.066667,0.933333,0.889706,0.639370,0.495798,0.223803,0.066667,0.703519,0.736439,41,1


In [181]:
X_train = df_train[features]
y_train = df_train['WinA']

In [180]:
df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,SeedDiff,WinRatioDiff,GapAvgDiff
0,2015_3106_3107,0.5,2015,3106,3107,0.933333,0.800000,0.083333,0.056501,0.541667,0.384505,0.566667,0.239130,0.316694
1,2015_3106_3110,0.5,2015,3106,3110,0.933333,0.866667,0.083333,0.056501,0.541667,0.207358,0.533333,0.239130,0.407843
2,2015_3106_3113,0.5,2015,3106,3113,0.933333,0.133333,0.083333,0.056501,0.713542,0.343837,0.900000,0.149457,0.337619
3,2015_3106_3114,0.5,2015,3106,3114,0.933333,0.666667,0.083333,0.056501,0.770833,0.386573,0.633333,0.119565,0.315630
4,2015_3106_3116,0.5,2015,3106,3116,0.933333,0.600000,0.083333,0.056501,0.205556,0.180337,0.666667,0.414493,0.421746


In [182]:
X_test = df_test[features]

In [206]:
# cross validation + model
kf = KFold(n_splits=4, random_state=0, shuffle=True)

clf = lgb.LGBMRegressor(n_estimator=500,metric='binary_logloss',objective='binary',)

def log_loss(y_true, y_pred):
    loss = log_loss(y_true, y_pred)
    return loss 

some_funcs ={
        'log_loss':make_scorer(log_loss)
    }
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

df_sample = pd.read_csv('WSampleSubmissionStage1.csv')
df_sample['Pred'] = y_pred
df_sample.to_csv('submmit_lgb.csv', index=False)